## In this notebook we run thought the data aquisitin pipeline to get derive results discussed in Samuel Heczko's masters thesis

In [29]:
import glob
import os
import connectome
import nibabel as nib
import pandas as pd
import regresson

import sys; sys.path
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
import imp


path = 'data/' ##cahnge for cluster




### 1. Get the connectomes from preregistered fmri images
In the thesis we largely conducted analysis on thethe Amsterdam Open MRI Collection (AOMIC). We used special preprocessed files and the data proveided by the AOIMIC people. We derive thea analysis in __ steps. All steps are done to all 900 subjects. We loop following proceedure over all atalases considererd.
1. Calcluate the time series data for each brain region for each atlas (extract_connectomes.py)
2. Compute the connectome of for each subject
3. For each atlas save the upper triangle of the connectivity matrix along the information which brain regions are connected in this connection.


Let's start with setting up the parameters

In [33]:
##defime global parameters
cluster = False
correlation_measure = 'correlation' #can be also tangent, partial
res_n = 1
res = f'{res_n}x{res_n}x{res_n}'
correlation_measure='correlation' ##set for calculation of the brain connectome, choose from 'correlation', 'tangent', 'partial' as implemented by nilearn
#templateICBM = '/kyb/agks/sheczko/Downloads/MastersThesis/code/data/templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii' ##use the ICBM T1 template


##add the data and names of thigs
if cluster:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*.nii') ##load up a subset of the subejct images

else:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*000*.nii') ##load up all the subejct images
subjects_idxs = []
for s_n in imgs_paths:
    subjects_idxs.append(s_n.split('_')[-1].split('.')[0]) #split the path to extract only the number of the subject from path



atlases = glob.glob(path + '/atlases/lawrance2021/label/Human/ICBM/*.nii.gz') ##get the atlases
anatomical_labels = glob.glob(path + '/atlases/lawrance2021/label/Human/Anatomical-labels-csv/*.csv') #get the anatomical labels (where available)
anatomical_label_names = []
for a_l in anatomical_labels:
    anatomical_label_names.append(a_l.split('/')[-1].split('.')[0]) #split the path to extract only the name of the atlas

colnames=['idx','anatomical_label'] 


##loop over atlases

Contiune with looping over the list of atlases and saving a csv wiht connectivity for all subejcts.

In [114]:
for atlas_path in atlases: ##loop over atlases
    atlas_name = (atlas_path.split('/')[-1].split('.')[0].split('_')[0]) #split the atlas path so we
    print(atlas_name)
    al_p = (any(n == atlas_name for n in anatomical_label_names)) ##find wheter we have the anatomical labellings for this atlas

    if al_p:
        anatomic_path = path + f'/atlases/lawrance2021/label/Human/Anatomical-labels-csv/' + atlas_name + '.csv'
        ana_labels = pd.read_csv(anatomic_path,names=colnames, header=None)
        ana_labels = ana_labels[ana_labels['idx'] != 0]
    else:
        ana_labels = None


    time_series = connectome.calculate_time_series(atlas_path = atlas_path,imgs_paths=imgs_paths)     ##get the time series of the from all subjects, using the atlas defined
    print(f'we have {len(time_series)} subjects and {time_series[0].shape} shaped time series')
    df_time_series = pd.DataFrame()
    
    ##save time series
    for array_i, array in enumerate(time_series):
        df_time_series = pd.concat([df_time_series,pd.DataFrame(data = array.flatten(), columns=[str(array_i)] )], axis = 1)
    
    if al_p:
        Brn_area_indecies = np.repeat(np.arange(len(ana_labels)),time_series[0].shape[0])
        df_labels = pd.DataFrame(data = ana_labels.iloc[Brn_area_indecies,1],columns=['anatomical_label'])
        df_labels = df_labels.reset_index()
        df_time_series = pd.concat([df_time_series,df_labels],axis = 1)
        


    
    df_time_series.to_csv(path_or_buf = path + f'/results/time_series/time_series_n_sub-{len(time_series)}_atlas-{atlas_name}.csv')



    

    #correlation_matrices, _ =  connectome.connectome(time_series = time_series,correlation_measure='correlation') #get the connectivity matrices

    #df_ = connectome.save_connectomes_df(correlation_matrices,anatomical_label_presence = al_p, anatomic_labels = ana_labels,path_to_save = path + 'results/', atlas_name = atlas_name, n_subjects = correlation_matrices.shape[0], correlation_measure = correlation_measure,subject_ixds = subjects_idxs)
    


DS00108
we have 8 subjects and (290, 105) shaped time series
DS06481
we have 8 subjects and (290, 6058) shaped time series
JHU
we have 8 subjects and (290, 48) shaped time series


IndexError: positional indexers are out-of-bounds

In [123]:
ana_labels.iloc[:,1].head()

0    Middle_Cerebellar_Peduncle
1        Pontine_Crossing_Tract
2          Corpus_Callosum_Genu
3       Body_of_corpus_callosum
4      Corpus_Callosum_Splenium
Name: anatomical_label, dtype: object

In [102]:
ana_labels

,idx,anatomical_label
0,1,Middle_Cerebellar_Peduncle
1,2,Pontine_Crossing_Tract
2,3,Corpus_Callosum_Genu
3,4,Body_of_corpus_callosum
4,5,Corpus_Callosum_Splenium
5,6,Fornix_Column+Body
6,7,R_Corticospinal_Tract
7,8,L_Corticospinal_Tract
8,9,R_Medial_Lemniscus
9,10,L_Medial_Lemniscus


In [89]:
anatomic_path = path + f'/atlases/lawrance2021/label/Human/Anatomical-labels-csv/' + 'JHU' + '.csv'
ana_labels = pd.read_csv(anatomic_path,names=colnames, header=None)
ana_labels = ana_labels[ana_labels['idx'] != 0]

In [93]:
np.repeat(np.arange(len(ana_labels)),time_series[0].shape[0])

array([ 0,  0,  0, ..., 47, 47, 47])

# find the regressons using the features extracted earlier
This part of the code is also regression_launcher.py
Now that we have extracted the features of our fMRI data, we are ready to apply the regression models to calcualte the predictability of our target variables. We subdivide this task into three steps:
1. loading and formatting our connectivity data
2. loading and formattin our target variables
3. fitting the desired model on the defined x and y

This process needs to be repeated over all the atlases and other variables we have been looping over. This allows us to asses the question which steps of the pipline architecture have the largest effect on the prediction accuracy.



In [28]:
imp.reload(regresson)


##let's find the paths to the the csv connectivity data we ahve
data_paths = glob.glob(path + '/results/connectomes/*.csv')

##load up regresors
regressors_df =  regresson.load_regressors(path + 'func_images/AOMIC/regressors/*.txt')
##choose the target variables, take as np arrays
GCA = regressors_df.regressor_iq.values
bmi = regressors_df.regressor_bmi.values

 #concatenate cognitive metrics into single variable
cognition = ['GCA','bmi']
#cognition = ['PMAT Correct', 'PMAT Response Time']
cog_metric = np.transpose(np.asarray([GCA, bmi]))

#set the number of permutations you want to perform
perm = 3
#set the number of cross-validation loops you want to perform
cv_loops = 5
#set the number of folds you want in the inner and outer folds of the nested cross-validation
k = 3
#set the proportion of data you want in your training set
train_size = .8
#set the number of variable you want to predict to be the number of variables stored in the cognition variablse
n_cog = np.size(cognition)
#set regression model type
regr = Ridge(fit_intercept = True, max_iter=1000000)
#set hyperparameter grid space you want to search through for the model
alphas = np.linspace(100, 1000, num=10, endpoint=True, dtype=None, axis=0)
#set y to be the cognitive metrics you want to predict. They are the same for every atlas (each subject has behavioural score regradless of parcellation)
Y = cog_metric


column_names_pred = []
column_names_real = []    


for perm_ixd in range(perm):
    for cog in cognition:
        column_names_pred.append(f'{cog}_perm_{perm_ixd + 1}_pred')
        column_names_real.append(f'{cog}_perm_{perm_ixd + 1}_real')
    





for data_path_i, data_path in enumerate(data_paths): ##loop over atlases
    

    current_path = data_paths[data_path_i + 1]
    current_atlas = current_path.split('/')[-1].split('_')[-1].split('.')[0]
    print(f'current atlas: ' + current_atlas)

    fc = regresson.load_data(current_path) ##set the imput variable to the current atlas connectome, gives subjects x features matrix

    print(f'data loaded:)')

    #set x data to be the input variable you want to use
    #ie fc, sc, or hc
    #cut the negative values from the correlations
    X = fc
    X[X<0] = 0


    #set the number of features 
    n_feat = X.shape[1]
    r2, preds, var, corr, featimp, cogtest,opt_alphas,n_pred = regresson.regression(X = X, Y = Y, perm = perm, cv_loops = cv_loops, k = k, train_size = 0.8, n_cog = n_cog, regr = regr, alphas = alphas, n_feat = n_feat,cognition = cognition)
    

        
    ##save data:

    df_preds = pd.DataFrame(preds.reshape(perm * n_cog,n_pred).T,columns = column_names_pred) ## we flatten the permutation axis 
    df_real = pd.DataFrame(cogtest.reshape(perm * n_cog,n_pred).T,columns = column_names_real)
    preds_real_df = pd.concat([df_preds,df_real],axis = 1, sort = True)


    result_r2 = pd.DataFrame(columns = [cog + '_r2' for cog in cognition], data = r2)
    result_var = pd.DataFrame(columns = [cog + '_var' for cog in cognition], data = var)
    opt_alphas_df = pd.DataFrame(columns = [cog + '_opt_alphas' for cog in cognition])
    result_df = pd.concat([result_var,result_r2],axis = 1)

    result_df.to_csv(path + f'results/ridge_regression/ridge_results_atlas-{current_atlas}.csv')
    preds_real_df.to_csv(path + f'results/ridge_regression/ridge_preds_atlas-{current_atlas}.csv')


current atlas: atlas-AAL
data loaded:)
Permutation 1
Cognition: GCA
Training Models


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.1s finished


1/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.2s finished


2/5 Complete


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.1s finished


4/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.2s finished


5/5 Complete
Testing Models
current atlas: atlas-DS00583
data loaded:)
Permutation 1
Cognition: GCA
Training Models


KeyboardInterrupt: 

In [26]:

column_names_pred = []
column_names_real = []    


for perm_ixd in range(perm):
    for cog in cognition:
        column_names_pred.append(f'{cog}_perm_{perm_ixd + 1}_pred')
        column_names_real.append(f'{cog}_perm_{perm_ixd + 1}_real')
    
    
##save data:

df_preds = pd.DataFrame(preds.reshape(perm * n_cog,n_pred).T,columns = column_names_pred) ## we flatten the permutation axis 
df_real = pd.DataFrame(cogtest.reshape(perm * n_cog,n_pred).T,columns = column_names_real)
preds_real_df = pd.concat([df_preds,df_real],axis = 1, sort = True)


result_r2 = pd.DataFrame(columns = [cog + '_r2' for cog in cognition], data = r2)
result_var = pd.DataFrame(columns = [cog + '_var' for cog in cognition], data = var)
opt_alphas_df = pd.DataFrame(columns = [cog + '_opt_alphas' for cog in cognition])
result_df = pd.concat([result_var,result_r2],axis = 1)

result_df.to_csv(path + f'results/ridge_regression/ridge_results_atlas-{current_atlas}.csv')
preds_real_df.to_csv(path + f'results/ridge_regression/ridge_preds_atlas-{current_atlas}.csv')

In [18]:
preds_real_df

,GCA_perm_1_pred,bmi_perm_1_pred,GCA_perm_2_pred,bmi_perm_2_pred,GCA_perm_3_pred,bmi_perm_3_pred,GCA_perm_4_pred,bmi_perm_4_pred,GCA_perm_5_pred,bmi_perm_5_pred,GCA_perm_1_real,bmi_perm_1_real,GCA_perm_2_real,bmi_perm_2_real,GCA_perm_3_real,bmi_perm_3_real,GCA_perm_4_real,bmi_perm_4_real,GCA_perm_5_real,bmi_perm_5_real
0,186.071931,24.135502,209.310151,24.111114,204.221377,25.525347,200.576436,24.449453,197.981595,22.436691,130.0,27.0,270.0,20.0,256.0,22.0,172.0,27.0,226.0,23.0
1,197.944629,24.089197,183.521658,24.904964,187.247643,23.118162,178.328069,20.558176,208.543834,23.564870,201.0,22.0,180.0,23.0,197.0,23.0,90.0,25.0,203.0,27.0
2,212.601445,23.133033,211.282000,22.215456,192.070488,28.609185,214.756526,22.858145,225.100928,28.963123,176.0,21.0,128.0,20.0,233.0,24.0,239.0,24.0,250.0,22.0
3,203.030371,28.911401,201.061282,23.613885,204.463873,24.207951,205.678831,22.870535,185.397177,23.550661,241.0,28.0,203.0,25.0,198.0,24.0,215.0,21.0,162.0,32.0
4,186.804065,21.850135,213.228944,19.950735,192.241515,38.701339,193.741199,23.144597,186.612066,26.570859,270.0,22.0,236.0,26.0,105.0,47.0,181.0,19.0,211.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,197.146468,26.278929,184.391343,20.200295,196.451454,20.574849,208.121654,22.250040,225.065444,24.279930,170.0,23.0,155.0,26.0,138.0,17.0,137.0,24.0,234.0,24.0
173,216.218427,23.045336,184.406719,22.745005,205.679850,23.701416,216.084516,23.234242,202.463100,26.533995,204.0,22.0,167.0,24.0,203.0,27.0,213.0,24.0,193.0,23.0
174,190.222103,22.590183,174.759577,23.119709,204.210510,26.498276,211.692251,21.314292,184.198595,20.567987,141.0,30.0,218.0,30.0,193.0,23.0,235.0,28.0,138.0,23.0
175,203.323110,21.747052,209.113547,22.437475,192.202434,22.161219,175.282334,24.518261,214.732986,27.596598,225.0,22.0,236.0,22.0,131.0,24.0,253.0,22.0,230.0,29.0


In [24]:
cogtest[4,0,:5]

array([226., 203., 250., 162., 211.])

In [16]:
r2 = np.random.random((5,2))
var = np.random.random((5,2))

In [29]:
print(r2)
print(np.logspace(start = 1, stop = 20000, num = 50, endpoint=True, dtype=None, axis=0))

[[0.08912571 0.33230529]
 [0.06010136 0.38565568]
 [0.07517595 0.49635083]
 [0.00390204 0.37530096]
 [0.03478023 0.51403657]]
[10. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf]


/kyb/agks/sheczko/.local/lib/python3.10/site-packages/numpy/core/function_base.py:284: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)


In [21]:
result_df

,GCA_r2,GCA_var,bmi_r2,bmi_var
0,0.352930,0.190132,0.946853,0.424036
1,0.834027,0.237094,0.935719,0.028854
2,0.292971,0.420059,0.623242,0.709984
3,0.492207,0.558587,0.191360,0.190099
4,0.442167,0.563885,0.067993,0.828950


In [ ]:
|